In [1]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

token = os.getenv("FourSquareID")
print(f"We have a Foursquare token: {token[0]}")

Python-dotenv could not parse statement starting at line 4
We have a Foursquare token: N


In [3]:
companies = list(db["companies"].find()) #{"name":1,"offices":1,"category_code":1}
companies[:5]

d</a> and <a href="http://www.xdrive.com" title="Xdrive" rel="nofollow">Xdrive</a>.</p>',
  'image': {'available_sizes': [[[150, 85],
     'assets/images/resized/0000/0823/823v1-max-150x150.jpg'],
    [[150, 85], 'assets/images/resized/0000/0823/823v1-max-250x250.jpg'],
    [[150, 85], 'assets/images/resized/0000/0823/823v1-max-450x450.jpg']],
   'attribution': None},
  'products': [{'name': 'Omnidrive', 'permalink': 'omnidrive'}],
  'relationships': [{'is_past': True,
    'title': 'Co-founder',
    'person': {'first_name': 'Geoff',
     'last_name': 'McQueen',
     'permalink': 'geoff-mcqueen'}},
   {'is_past': True,
    'title': 'Founder & CEO',
    'person': {'first_name': 'Nik',
     'last_name': 'Cubrilovic',
     'permalink': 'nik-cubrilovic'}},
   {'is_past': True,
    'title': '',
    'person': {'first_name': 'Tony',
     'last_name': 'Guntharp',
     'permalink': 'tony-guntharp'}},
   {'is_past': True,
    'title': 'Board of Advisors',
    'person': {'first_name': 'Mike',
    

In [4]:
import pandas as pd
df = pd.DataFrame(companies)
display(df.head())

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,video_embeds,screenshots,external_links,partners,founded_month,founded_day,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],10.0,17.0,NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d8f,Omnidrive,omnidrive,http://www.crunchbase.com/company/omnidrive,http://www.omnidrive.com,http://www.omnidrive.com/blog,http://feeds.feedburner.com/omnidrive,Nomadesk,network_hosting,NaN,...,[],[],[],[],11.0,1.0,9.0,15.0,,None
3,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,"[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],9.0,15.0,NaN,NaN,NaN,NaN
4,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,[],[],[],[],6.0,2.0,NaN,NaN,None,None


In [5]:
df = df.explode('offices')
dfOfficeData = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
cleanData = pd.concat([df,dfOfficeData], axis=1)


In [6]:
cleanData = cleanData.drop(columns=["_id","offices"])

In [8]:
import math
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None
        

cleanData["location"] = cleanData[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
cleanData[["latitude","longitude","location"]].head()

,latitude,longitude,location
0,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
1,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
2,NaN,NaN,None
3,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."


In [7]:
#cleanData = cleanData.drop(columns=["latitude","longitude"])


## Resolving duplicate column

We ran into an issue with transforming our dataframe back to a json - one of the columns is a duplicate. We can rename it to avoid the issue.

In [9]:
colscount = {item:list(cleanData.columns).count(item) for item in list(cleanData.columns)}
[e for e in colscount if colscount[e] > 1] #this code lets us find the duplicate column, which is 'description'

['description']

In [10]:
cleanData['description'] #We are giving the descriptions 1 and 2 numbers in order to avoid the error with duplicate columns.
cols = []
count = 1
for column in cleanData.columns:
    if column == 'description':
        cols.append(f'description_{count}')
        count+=1
        continue
    cols.append(column)
cleanData.columns = cols

In [11]:
cleanData.head()

,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,...,description_2,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,location
0,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,1996.0,...,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,2005.0,...,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
1,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,2005.0,...,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
2,Omnidrive,omnidrive,http://www.crunchbase.com/company/omnidrive,http://www.omnidrive.com,http://www.omnidrive.com/blog,http://feeds.feedburner.com/omnidrive,Nomadesk,network_hosting,NaN,2005.0,...,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN,None
3,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,2005.0,...,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."


In [12]:
# When exporting data for mongoimport, REMEMBER! -> orient=records
cleanData.to_json("../source/cleaned_offices.json", orient="records")

# Next step: 
# mongoimport --db companies --collection companies_wlocation --jsonArray --drop cleaned_offices.json

## Create geospartial index
# Indexes > Create Index > (write any indexname) > Select fieldname:"location" and 2dsphere
